In [24]:
import numpy as np
import scipy.linalg as sln
import matplotlib.pyplot as plt
import warnings
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

Points
<br># -> Doc(Main)
<br>## -> My Notes
<br>### -> Missing, Corrections to do

In [25]:
def mv(A, v, transp_flag):
    if transp_flag == 0:
        return A @ v
    else:
        return A.T @ v

In [26]:
def unv(j, n):
    e = np.zeros(n)
    e[j] = 1
    return e

In [27]:
###Get rid of function aspect
def element(A, i=None, j=None, n=None):
    # Handling default arguments
    if i is None:
        i = slice(None)  # Selecting all rows
    if j is None:
        j = slice(None)  # Selecting all columns

    if isinstance(A, np.ndarray):  # Matrix
        if min(A.shape) > 1:  # Check if A is not a vector
            if isinstance(i, list) and isinstance(j, list):
                e = A[np.ix_(i, j)]
            else:
                e = A[i, j]
        else:
            e = A[i] if isinstance(i, list) else A[i]
    else:  # Function
        if j is None:
            raise ValueError("j has to be nonempty when A is a function")
        e = mv(A, unv(j, n), 0)
        if i is not None:
            e = e[i]

    return e

In [28]:
def krylov_ata(A, v1=None, k=10, full=1, reortho=2):
    if v1 is None:
        v1 = np.random.randn(A.shape[1])

    k = min(k, min(A.shape))

    alpha = np.zeros(k)
    beta = np.zeros(k if full else k-1)

    len_v1 = len(v1)
    if reortho:
        V = np.zeros((len_v1, k + 1))
        V[:, 0] = v1 / np.linalg.norm(v1)
        #U = np.zeros((A.shape[0], k))
    else:
        v = v1 / np.linalg.norm(v1)

    for j in range(k):
        if reortho:
            r = mv(A, V[:, j], 0)
            if j == 0 and reortho == 2:
                U = np.zeros((len(r), k))
        else:
            r = mv(A, v, 0)

        if j > 0:
            if reortho == 2:
                r -= beta[j-1] * U[:, j-1]
                r -= U[:, :j] @ (U[:, :j].T @ r)
            else:
                r -= beta[j-1] * u
        alpha[j] = np.linalg.norm(r)
        if alpha[j] == 0:
            break

        if reortho == 2:
            U[:, j] = r / alpha[j]
            r = mv(A, U[:, j], 1)
        else:
            u = r / alpha[j]
            r = mv(A, u, 1)

        if reortho:
            r -= alpha[j] * V[:, j]
            r -= V[:, :j+1] @ (V[:, :j+1].T @ r)
        else:
            r -= alpha[j] * v

        if j < k - 1 or full:
            beta[j] = np.linalg.norm(r)
            if beta[j] == 0:
                break

            if reortho:
                V[:, j+1] = r / beta[j]
            else:
                v = r / beta[j]

    if not reortho:
        V = v
    if reortho < 2:
        U = u

    
    return V, U, alpha, beta

In [29]:
def krylov_ata_expand(A, V, U, c, k=10):
    m = V.shape[1]
    V = np.concatenate((V, np.zeros((V.shape[0], k))), axis=1)
    U = np.concatenate((U, np.zeros((U.shape[0], k))), axis=1)
    alpha = np.zeros(k)
    beta = np.zeros(k)
    
    for j in range(m, k + m):
        if j == m:
            r = mv(A, V[:, j - 1], 0) - (U[:, :j - 1] @ c.T)
        else:
            r = mv(A, V[:, j - 1], 0) - beta[j - m - 1] * U[:, j - 2]

        r -= - U[:, :j - 1] @ (U[:, :j - 1].T @ r)
        alpha[j - m] = np.linalg.norm(r)
        if alpha[j - m] == 0:
            break
        U[:, j - 1] = r / alpha[j - m]
        r = mv(A, U[:, j - 1], 1) - alpha[j - m] * V[:, j - 1]
        r -= V[:, :j] @ (V[:, :j].T @ r)
        beta[j - m] = np.linalg.norm(r)
        if beta[j - m] == 0:
            break
        V[:, j] = r / beta[j - m]

    return V, U, alpha, beta

In [34]:
def krylov_schur_svd(A, **kwargs):
    nr = kwargs.get('nr', 1)
    v1 = kwargs.get('v1', None)
    tol = kwargs.get('tol', 1e-6)
    absrel = kwargs.get('absrel', 'rel')
    mindim = kwargs.get('mindim', 10)
    maxdim = kwargs.get('maxdim', 20)
    maxit = kwargs.get('maxit', 1000)
    target = kwargs.get('target', np.inf)
    info = kwargs.get('info',1)

    if v1 is None:
        v1 = np.random.rand(A.shape[1])
           
    if mindim < nr:
        mindim = nr
    if maxdim < 2 * mindim:
        maxdim = 2 * mindim
    
    if absrel == 'rel' and np.issubdtype(A.dtype, np.number):
        tol = tol * np.linalg.norm(A, 1)

    B = np.zeros((maxdim, maxdim + 1))
    V, U, alpha, beta = krylov_ata(A, v1, mindim)
    
    # Bidiagonal Form for the first mindim rows and cols
    B[:mindim + 1, :mindim + 1] = np.diag(np.append(alpha, [0])) + np.diag(beta, 1)

    hist = np.zeros(maxit, dtype=np.float64)
    np.set_printoptions(precision=8)
    
    for k in range(maxit):
        V, U, alpha, beta = krylov_ata_expand(A, V, U, B[:mindim, mindim], maxdim - mindim)
        B[mindim: maxdim, mindim: maxdim] = np.diag(alpha) + np.diag(beta[:maxdim - mindim - 1], 1)
        
        B[maxdim - 1, maxdim] = beta[maxdim - mindim - 1]
        X, sigma, Y = np.linalg.svd(B[:maxdim, :maxdim])
        
        # Restart of Lanczos algorithm
        V = np.concatenate((element(V[:, :maxdim] @ Y, list(range(V.shape[0])), list(range(mindim))), V[:, maxdim:maxdim + 1]), axis=1)
        U = element(U[:, :maxdim] @ X, list(range(U.shape[0])), list(range(mindim)))
        
        c = B[:, maxdim]
        e = (c @ X)[:mindim]

        B[:mindim, :mindim + 1] = np.concatenate((np.diag(sigma[:mindim]), e.reshape(-1, 1)), axis=1)
        err = np.linalg.norm(e[:nr])
        hist[k] = err
        
        if info:
            print(str(k) + ": " + str(hist[k]))
        
        if hist[k] < tol:
            sigma = sigma[:nr]
            V = V[:, :nr]
            U = U[:, :nr]
            mvs = np.arange(1, k + 1) * (maxdim - mindim) + mindim
            print(f"Found after {k} iterations with residual = {hist[k]}")
            return sigma, V, U, hist[:k+1], mvs
    
    mvs = 2 * (np.arange(1, k + 1) * (maxdim - mindim) + mindim)
    if info:
        print(f"Quit after max {k} iterations with residual = {err}")
    sigma = sigma[:mindim]
    V = V[:, :mindim]
    return sigma, V, U, hist, mvs

Tests

In [32]:
A = np.random.rand(200,100)
A

array([[ 0.4189,  0.2883,  0.8791, ...,  0.2089,  0.1330,  0.9720],
       [ 0.8940,  0.8177,  0.9402, ...,  0.6509,  0.1549,  0.9568],
       [ 0.9516,  0.6285,  0.6579, ...,  0.0546,  0.4450,  0.4294],
       ...,
       [ 0.4985,  0.7463,  0.6161, ...,  0.3733,  0.4859,  0.5512],
       [ 0.7652,  0.7560,  0.1963, ...,  0.7100,  0.8250,  0.6307],
       [ 0.3147,  0.1564,  0.4410, ...,  0.4126,  0.0246,  0.7984]])

In [35]:
sigma, V, U, hist, mvs = krylov_schur_svd(A, **{'info' : 2, 'tol' : 1e-200, 'maxit': 7, 'nr' : 3})

0: 0.028980390099821597
1: 0.0008192285864678848
2: 1.695346194602129e-07
3: 8.557581065326106e-12
4: 1.3922364584763386e-15
5: 0.0
Found after 5 iterations with residual = 0.0


In [23]:
hist

array([1.08308967e-01, 4.95101308e-04, 2.53482434e-08, 2.39907808e-12,
       3.44234363e-16, 0.00000000e+00])

In [260]:
np.linalg.norm(A)

np.float64(2.0017916254222134)

In [265]:
v = [ 0.0000,  0.6388,  0.6530,  0.7064,  1.1590,  1.6503,  1.8708,  2.3015,  2.5898,
  2.8051]

In [266]:
v[:3]

[0.0, 0.6388, 0.653]

In [267]:
np.linalg.norm(v[:3])

np.float64(0.9134957252226198)

# Mega Tester for finding ATA_Expand Error

In [148]:
B = np.random.rand(10,7)

In [270]:
def tester(A, **kwargs):
    nr = kwargs.get('nr', 1)
    v1 = kwargs.get('v1', None)
    tol = kwargs.get('tol', 1e-6)
    absrel = kwargs.get('absrel', 'rel')
    mindim = kwargs.get('mindim', 10)
    maxdim = kwargs.get('maxdim', 20)
    maxit = kwargs.get('maxit', 1000)
    target = kwargs.get('target', np.inf)
    info = kwargs.get('info',1)

    if v1 is None:
        v1 = np.random.rand(A.shape[1])
    
    if mindim < nr:
        mindim = nr
    if maxdim < 2 * mindim:
        maxdim = 2 * mindim
    
    if absrel == 'rel' and np.issubdtype(A.dtype, np.number):
        tol = tol * np.linalg.norm(A, 1)

    B = np.zeros((maxdim, maxdim + 1))
    V, U, alpha, beta = krylov_ata(A, v1, mindim)
    #print(alpha)
    #print(beta)
    
    # Bidiagonal Form
    B[:mindim + 1, :mindim + 1] = np.diag(np.append(alpha, [0])) + np.diag(beta, 1)
    print(1)
    #print(B)
    
    hist = np.zeros(maxit)
    
    print(1)
    print(A)
    print(2)
    print(V)
    print(3)
    print(U)
    print(4)
    print(B[:mindim, mindim])
    print(5)
    print(maxdim - mindim)
    #V, U, alpha, beta = krylov_ata_expand(A, V, U, B[:mindim, mindim], maxdim - mindim)
    #B[mindim: maxdim, mindim: maxdim] = np.diag(alpha) + np.diag(beta[:maxdim - mindim - 1], 1)
    print()    
    return A,V,U,B[:mindim, mindim], maxdim - mindim     

In [271]:
a,v,u,b,d = tester(B, **{'info' : 2, 'tol' : 1e-200, 'maxit': 7, 'nr' : 1,'maxdim': 15, 'mindim': 5})

1
1
[[ 0.6276  0.0101  0.2266  0.5146  0.8531  0.6498  0.3278]
 [ 0.6545  0.8964  0.6144  0.9838  0.0234  0.0572  0.9396]
 [ 0.1042  0.4513  0.2136  0.5894  0.0550  0.5751  0.7596]
 [ 0.5582  0.3725  0.8571  0.4183  0.7060  0.8153  0.9578]
 [ 0.3397  0.3533  0.6166  0.6457  0.0439  0.7034  0.2721]
 [ 0.8103  0.0991  0.5226  0.5397  0.2915  0.4921  0.6187]
 [ 0.8362  0.0212  0.2194  0.7007  0.8103  0.9069  0.3109]
 [ 0.0126  0.8889  0.5931  0.6955  0.6427  0.1970  0.4673]
 [ 0.7118  0.0275  0.2076  0.1468  0.4754  0.8085  0.8875]
 [ 0.8360  0.9918  0.5109  0.6168  0.7391  0.0533  0.4362]]
2
[[ 0.4863 -0.0840  0.3608  0.1116 -0.0500 -0.4357]
 [ 0.4435 -0.2001 -0.4965 -0.3421  0.2707 -0.4732]
 [ 0.4248 -0.1065  0.2396 -0.5761  0.1214  0.6321]
 [ 0.2472  0.4810 -0.6063  0.2591  0.1640  0.3079]
 [ 0.5034 -0.2596  0.0977  0.6496 -0.1027  0.2380]
 [ 0.1005  0.6296  0.4344  0.0496  0.5356 -0.1515]
 [ 0.2484  0.4965 -0.0001 -0.2167 -0.7650 -0.1107]]
3
[[ 0.2794  0.0192  0.2689  0.4817 -0.0697]


In [222]:
def krylov_ata_expand(A, V, U, c, k=10):
    m = V.shape[1]
    V = np.concatenate((V, np.zeros((V.shape[0], k))), axis=1)
    U = np.concatenate((U, np.zeros((U.shape[0], k))), axis=1)
    alpha = np.zeros(k)
    beta = np.zeros(k)
    
    for j in range(m , k + m):
        print(A.shape)
        print(V[:, j - 1].shape)
        print((A @ V[:, j - 1]).shape)
        print(U[:, : j - 1].shape)
        print(c.shape)
        if j == m:
            
            ###Either c or c.T
            r = mv(A, V[:, j - 1], 0) - (U[:, : j - 1] @ c)
        else:
            r = mv(A, V[:, j - 1], 0) - beta[j - m - 1] * U[:, j - 2]
        
        print(j)
        print(r)
        print('######')
        
        r = r - U[:, : j - 1] @ (U[:, : j - 1].T @ r)
        alpha[j - m] = np.linalg.norm(r)
        if alpha[j - m] == 0:
            break
        U[:, j - 1] = r / alpha[j - m]
        r = mv(A, U[:, j - 1], 1) - alpha[j - m] * V[:, j - 1]
        r = r - V[:, : j] @ (V[:, :j].T @ r)
        beta[j - m] = np.linalg.norm(r)
        if beta[j - m] == 0:
            break
        V[:, j] = r / beta[j - m]

    return V, U, alpha, beta

In [223]:
V, U, alpha, beta = krylov_ata_expand(a,v,u,b,d)

(10, 7)
(7,)
(10,)
(10, 5)
(5,)
6
[ 0.1260  0.1113  0.0360  0.0123 -0.1926  0.0613  0.0068  0.0976 -0.0745
 -0.1923]
######
(10, 7)
(7,)
(10,)
(10, 6)
(5,)
7
[-0.0853 -0.1211 -0.0468  0.3407 -0.3765 -0.0952 -0.3029  0.0816  0.3300
  0.1366]
######
(10, 7)
(7,)
(10,)
(10, 7)
(5,)
8
[-0.3642 -1.1516 -0.7048 -0.7886 -0.9251 -0.7490 -0.6399 -0.5717 -0.5663
 -0.8537]
######
(10, 7)
(7,)
(10,)
(10, 8)
(5,)
9
[-0.5670 -1.9628 -1.5160 -1.1942 -0.9251 -1.1546 -1.4511 -0.9772 -0.9719
 -0.4481]
######
(10, 7)
(7,)
(10,)
(10, 9)
(5,)
10
[-2.5500 -5.9764 -5.8826 -3.0223 -1.2282 -3.3312 -4.7288 -2.8785 -3.0765
  1.4634]
######
(10, 7)
(7,)
(10,)
(10, 10)
(5,)
11
[-16.1907 -36.9962 -39.0232 -17.4580 -3.0493 -19.8612 -31.3265 -17.7540
 -19.1743  16.4009]
######
(10, 7)
(7,)
(10,)
(10, 11)
(5,)
12
[-199.0360 -438.9395 -470.6434 -203.4000 -28.5324 -235.1074 -371.3925
 -209.8497 -228.3725  209.3639]
######
(10, 7)
(7,)
(10,)
(10, 12)
(5,)
13
[-3930.4026 -8696.5451 -9328.9129 -4028.0007 -544.3138 -4652.82

In [224]:
alpha

array([ 0.3502,  0.7190,  0.0000,  3.2754,  23.0619,  225.4854,
        3608.4118,  89588.0324,  3205165.3986,  156215235.2676])

In [225]:
beta

array([ 0.0392,  0.0000,  1.6846,  10.0163,  74.3784,  908.8731,
        18050.7222,  537538.5906,  22436169.9188,  1249721896.0021])

In [226]:
def krylov_ata_expand(A, V, U, c, k=10):
    m = V.shape[1]
    V = np.concatenate((V, np.zeros((V.shape[0], k))), axis=1)
    U = np.concatenate((U, np.zeros((U.shape[0], k))), axis=1)
    alpha = np.zeros(k)
    beta = np.zeros(k)
    
    for j in range(m, k + m):
        if j == m:
            r = mv(A, V[:, j - 1], 0) - (U[:, :j - 1] @ c.T)
        else:
            r = mv(A, V[:, j - 1], 0) - beta[j - m - 1] * U[:, j - 2]

        r = r - U[:, :j - 1] @ (U[:, :j - 1].T @ r)
        alpha[j - m] = np.linalg.norm(r)
        if alpha[j - m] == 0:
            break
        U[:, j - 1] = r / alpha[j - m]
        r = mv(A, U[:, j - 1], 1) - alpha[j - m] * V[:, j - 1]
        r = r - V[:, :j] @ (V[:, :j].T @ r)
        beta[j - m] = np.linalg.norm(r)
        if beta[j - m] == 0:
            break
        V[:, j] = r / beta[j - m]

    return V, U, alpha, beta

In [227]:
V, U, alpha, beta = krylov_ata_expand(a,v,u,b,d)

In [228]:
alpha

array([ 0.3502,  0.7190,  0.0000,  3.2754,  23.0619,  225.4854,
        3608.4118,  89588.0324,  3205165.3986,  156215235.2676])

In [229]:
beta

array([ 0.0392,  0.0000,  1.6846,  10.0163,  74.3784,  908.8731,
        18050.7222,  537538.5906,  22436169.9188,  1249721896.0021])